### Questions:
Recently our director is interested to find out if we could predict our revenue based on account managers’ activies. 
Some of his questions are below:

1. Can we find any good predictor for number of Closures or % fee collection using account manager activities in given dataset?	
2. Are account manager’s activities a good indicator for revenue?
3. Is floating interest a good indicator for revenue?
4. Does transaction fee impact revenue?


### Import required libraries

In [800]:
# Part 1
import pandas as pd
import numpy as np
# Part 2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Part 3
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Step 1: Extract - Transform - Loading

## Data extraction

In [137]:
# Read data file revenue.csv which download from company dashboard with pandas
df=pd.read_csv('revenue.csv')

In [138]:
# check the data quickly to make sure it is the right file
df.tail()

,Year,Month,A Status Account Count,Annual Review Month Unique Count,Service Month Unique Count,TFG,Fee Collected %,Frequent Trading Customer Partner Count,Activity Email Count,Activity Left Message Count,...,Sales Volume,TV Transaction Processed Count,Closure Count,Billable Month Volume,Traded100 Account Count,Traded1000 Account Count,Traded200 Account Count,Traded500 Account Count,Trading Customer Partner Count,Closing Available Balance
329,NaN,May,5073,12,2576,3.192582e+05,0.620230,3494,551.0,968.0,...,2.232797e+06,7394,19.0,4.507436e+06,1773,802,1521,1119,5267,9.014108e+07
330,NaN,June,5072,39,2460,4.434937e+05,0.738423,4260,466.0,715.0,...,3.067886e+06,10369,27.0,6.135884e+06,1985,978,1742,1351,6841,8.947853e+07
331,NaN,July,5057,75,2643,5.756627e+05,0.816795,5305,597.0,951.0,...,3.928258e+06,12323,30.0,8.053953e+06,2097,1151,1907,1522,7997,8.926536e+07
332,2020 Total,NaN,60980,361,16316,2.816847e+06,0.598382,31591,3301.0,6102.0,...,1.945326e+07,70167,NaN,3.968876e+07,13799,6629,12112,9235,46898,7.225113e+08
333,Grand Total,NaN,1862464,34337,691530,2.326754e+08,0.754963,3230770,161763.0,248038.0,...,1.830398e+09,7842432,21447.0,3.736002e+09,976766,593887,903759,755072,5341020,2.013616e+10


In [139]:
# check all columns we have
df.columns

Index(['Year', 'Month', 'A Status Account Count',
       'Annual Review Month Unique Count', 'Service Month Unique Count', 'TFG',
       'Fee Collected %', 'Frequent Trading Customer Partner Count',
       'Activity Email Count', 'Activity Left Message Count',
       'Annual Review Count', 'Service Count', 'Visit Drop In Count',
       'Fair Trading Strike Count', 'Visit Service Count', 'Search Count',
       'Search No Match Count', 'Purchase Volume', 'Sales Volume',
       'TV Transaction Processed Count', 'Closure Count',
       'Billable Month Volume', 'Traded100 Account Count',
       'Traded1000 Account Count', 'Traded200 Account Count',
       'Traded500 Account Count', 'Trading Customer Partner Count',
       'Closing Available Balance'],
      dtype='object')

In [140]:
# check datatype
df.dtypes

Year                                        object
Month                                       object
A Status Account Count                       int64
Annual Review Month Unique Count             int64
Service Month Unique Count                   int64
TFG                                        float64
Fee Collected %                            float64
Frequent Trading Customer Partner Count      int64
Activity Email Count                       float64
Activity Left Message Count                float64
Annual Review Count                        float64
Service Count                              float64
Visit Drop In Count                        float64
Fair Trading Strike Count                  float64
Visit Service Count                        float64
Search Count                               float64
Search No Match Count                      float64
Purchase Volume                            float64
Sales Volume                               float64
TV Transaction Processed Count 

## Data transform

Datatype is looking good. However we can see our dataset doesn't have proper datetime columns. So we will go ahead and construct a proper datetime columns by filling empty year. Then we will need to remove duplicated row (total row end of each year)



In [141]:
# Contruct all missing year value by using fill NaN with previous row value
df['Year']=df['Year'].fillna(method='ffill')

In [142]:
# remove all total columns
df=df[~df.Year.str.contains("Total")]

In [143]:
# construct year- month columns
df['date']=df['Month'] + " - " + df['Year']
df['date']=pd.to_datetime(df['date'])

In [144]:
# check our new data
df.tail(5)

,Year,Month,A Status Account Count,Annual Review Month Unique Count,Service Month Unique Count,TFG,Fee Collected %,Frequent Trading Customer Partner Count,Activity Email Count,Activity Left Message Count,...,TV Transaction Processed Count,Closure Count,Billable Month Volume,Traded100 Account Count,Traded1000 Account Count,Traded200 Account Count,Traded500 Account Count,Trading Customer Partner Count,Closing Available Balance,date
327,2020,March,5112,69,2413,459012.3525,0.627050,4849,477.0,767.0,...,11069,23.0,6509895.810,2095,1011,1879,1438,7434,90783901.22,2020-03-01
328,2020,April,5015,6,2269,170373.1300,0.766216,1892,423.0,919.0,...,3630,20.0,2298004.410,1215,482,1029,728,2772,90527026.22,2020-04-01
329,2020,May,5073,12,2576,319258.2300,0.620230,3494,551.0,968.0,...,7394,19.0,4507435.560,1773,802,1521,1119,5267,90141084.02,2020-05-01
330,2020,June,5072,39,2460,443493.7400,0.738423,4260,466.0,715.0,...,10369,27.0,6135883.593,1985,978,1742,1351,6841,89478534.02,2020-06-01
331,2020,July,5057,75,2643,575662.7375,0.816795,5305,597.0,951.0,...,12323,30.0,8053952.985,2097,1151,1907,1522,7997,89265359.03,2020-07-01


Next, we will go ahead and check for all columns with NaN values


In [145]:
# check for any columns with NaN value
df.loc[:, df.isna().any()]

,Activity Email Count,Activity Left Message Count,Annual Review Count,Service Count,Visit Drop In Count,Fair Trading Strike Count,Visit Service Count,Search Count,Search No Match Count,Closure Count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0
...,...,...,...,...,...,...,...,...,...,...
327,477.0,767.0,70.0,3028.0,28.0,0.0,253.0,56236.0,3882.0,23.0
328,423.0,919.0,6.0,2713.0,0.0,0.0,9.0,11293.0,1189.0,20.0
329,551.0,968.0,12.0,3278.0,7.0,0.0,68.0,57704.0,3287.0,19.0
330,466.0,715.0,39.0,3281.0,22.0,0.0,220.0,90247.0,4702.0,27.0


In [146]:
# Loop through all columns and calculate total missing values NaN each columns
for col in df.columns:
    print (col,' has a total of ',df[col].isna().sum(),' NaN values')

Year  has a total of  0  NaN values
Month  has a total of  0  NaN values
A Status Account Count  has a total of  0  NaN values
Annual Review Month Unique Count  has a total of  0  NaN values
Service Month Unique Count  has a total of  0  NaN values
TFG  has a total of  0  NaN values
Fee Collected %  has a total of  0  NaN values
Frequent Trading Customer Partner Count  has a total of  0  NaN values
Activity Email Count  has a total of  62  NaN values
Activity Left Message Count  has a total of  62  NaN values
Annual Review Count  has a total of  62  NaN values
Service Count  has a total of  62  NaN values
Visit Drop In Count  has a total of  62  NaN values
Fair Trading Strike Count  has a total of  62  NaN values
Visit Service Count  has a total of  62  NaN values
Search Count  has a total of  250  NaN values
Search No Match Count  has a total of  250  NaN values
Purchase Volume  has a total of  0  NaN values
Sales Volume  has a total of  0  NaN values
TV Transaction Processed Count  h

In [147]:
# Get % of missing value over total rows
print (df['Search No Match Count'].isna().values.sum()/len(df))

0.8143322475570033


As we can see Search columns has the highest number of NaN due to this tool only came availalbe in 2016.
In this study, we will decide to remove this feature due to more than 80% of values are missing.


In [148]:
# delete this feature
del df['Search Count']
del df['Search No Match Count']

In [149]:
# Loop through all columns and calculate total missing values NaN each columns
for col in df.columns:
    print (col,' has a total of ',df[col].isna().sum(),' NaN values')

Year  has a total of  0  NaN values
Month  has a total of  0  NaN values
A Status Account Count  has a total of  0  NaN values
Annual Review Month Unique Count  has a total of  0  NaN values
Service Month Unique Count  has a total of  0  NaN values
TFG  has a total of  0  NaN values
Fee Collected %  has a total of  0  NaN values
Frequent Trading Customer Partner Count  has a total of  0  NaN values
Activity Email Count  has a total of  62  NaN values
Activity Left Message Count  has a total of  62  NaN values
Annual Review Count  has a total of  62  NaN values
Service Count  has a total of  62  NaN values
Visit Drop In Count  has a total of  62  NaN values
Fair Trading Strike Count  has a total of  62  NaN values
Visit Service Count  has a total of  62  NaN values
Purchase Volume  has a total of  0  NaN values
Sales Volume  has a total of  0  NaN values
TV Transaction Processed Count  has a total of  0  NaN values
Closure Count  has a total of  1  NaN values
Billable Month Volume  has 

In [150]:
# Get % of missing value over total rows
print (df['Activity Email Count'].isna().values.sum()/len(df))

0.20195439739413681


The next features has 21% of values missing is Account manager activities. This can be explained by in the first 5 years from 1995 to 2000, the company hasn't got the KPI system for account manager and it only became available in 2000.
In this study, we will not taking the approach to replace missing values by mean() or median of this feature due to the fact that in the first few years, data quality might not be as accurate as after 2000. So we will remove this NaN value and only study from 2000 towards.

In [151]:
# check our data before remove all NaN values
df.tail()

,Year,Month,A Status Account Count,Annual Review Month Unique Count,Service Month Unique Count,TFG,Fee Collected %,Frequent Trading Customer Partner Count,Activity Email Count,Activity Left Message Count,...,TV Transaction Processed Count,Closure Count,Billable Month Volume,Traded100 Account Count,Traded1000 Account Count,Traded200 Account Count,Traded500 Account Count,Trading Customer Partner Count,Closing Available Balance,date
327,2020,March,5112,69,2413,459012.3525,0.627050,4849,477.0,767.0,...,11069,23.0,6509895.810,2095,1011,1879,1438,7434,90783901.22,2020-03-01
328,2020,April,5015,6,2269,170373.1300,0.766216,1892,423.0,919.0,...,3630,20.0,2298004.410,1215,482,1029,728,2772,90527026.22,2020-04-01
329,2020,May,5073,12,2576,319258.2300,0.620230,3494,551.0,968.0,...,7394,19.0,4507435.560,1773,802,1521,1119,5267,90141084.02,2020-05-01
330,2020,June,5072,39,2460,443493.7400,0.738423,4260,466.0,715.0,...,10369,27.0,6135883.593,1985,978,1742,1351,6841,89478534.02,2020-06-01
331,2020,July,5057,75,2643,575662.7375,0.816795,5305,597.0,951.0,...,12323,30.0,8053952.985,2097,1151,1907,1522,7997,89265359.03,2020-07-01


In [152]:
# Remove all NaN value
df = df[~df.isna().any(axis=1)]

In [153]:
# check final data shape
df.shape

(245, 27)

In [154]:
# Now we check for first 5 and bottom 5 rows of dataframe
df.head()

,Year,Month,A Status Account Count,Annual Review Month Unique Count,Service Month Unique Count,TFG,Fee Collected %,Frequent Trading Customer Partner Count,Activity Email Count,Activity Left Message Count,...,TV Transaction Processed Count,Closure Count,Billable Month Volume,Traded100 Account Count,Traded1000 Account Count,Traded200 Account Count,Traded500 Account Count,Trading Customer Partner Count,Closing Available Balance,date
66,2000,February,3345,0,0,512760.98,0.795961,7258,0.0,0.0,...,17907,57.0,9.303623e+06,2243,1424,2110,1786,12592,24587851.24,2000-02-01
68,2000,April,3343,0,0,489407.27,0.751309,7448,6.0,0.0,...,17209,30.0,8.906479e+06,2306,1489,2179,1828,12628,26308623.30,2000-04-01
69,2000,May,3591,0,0,705151.59,0.795144,8188,35.0,0.0,...,22837,64.0,1.176434e+07,2529,1776,2407,2119,15858,28129949.79,2000-05-01
70,2000,June,3703,0,0,599442.24,0.790643,6694,18.0,0.0,...,21423,57.0,1.042658e+07,2472,1617,2323,2002,15404,28490604.70,2000-06-01
71,2000,July,4148,0,0,705770.43,0.765749,9104,10.0,0.0,...,21941,50.0,1.180272e+07,2778,1776,2606,2232,16021,29399982.36,2000-07-01


In [163]:
# check last 5 rows
df.tail()

,Year,Month,A Status Account Count,Annual Review Month Unique Count,Service Month Unique Count,TFG,Fee Collected %,Frequent Trading Customer Partner Count,Activity Email Count,Activity Left Message Count,...,TV Transaction Processed Count,Closure Count,Billable Month Volume,Traded100 Account Count,Traded1000 Account Count,Traded200 Account Count,Traded500 Account Count,Trading Customer Partner Count,Closing Available Balance,date
327,2020,March,5112,69,2413,459012.3525,0.627050,4849,477.0,767.0,...,11069,23.0,6509895.810,2095,1011,1879,1438,7434,90783901.22,2020-03-01
328,2020,April,5015,6,2269,170373.1300,0.766216,1892,423.0,919.0,...,3630,20.0,2298004.410,1215,482,1029,728,2772,90527026.22,2020-04-01
329,2020,May,5073,12,2576,319258.2300,0.620230,3494,551.0,968.0,...,7394,19.0,4507435.560,1773,802,1521,1119,5267,90141084.02,2020-05-01
330,2020,June,5072,39,2460,443493.7400,0.738423,4260,466.0,715.0,...,10369,27.0,6135883.593,1985,978,1742,1351,6841,89478534.02,2020-06-01
331,2020,July,5057,75,2643,575662.7375,0.816795,5305,597.0,951.0,...,12323,30.0,8053952.985,2097,1151,1907,1522,7997,89265359.03,2020-07-01
